In [1]:
# https://python.langchain.com/docs/integrations/tools/
# https://platform.openai.com/docs/guides/moderation

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [6]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

response = client.moderations.create(
    model="text-moderation-latest",
    input='I want to kill my neighbor'
    #input='qual a capital do Brasil ?'
)

if response:
    print("Model: ", response.model)
    results = response.results
    if results:
        for result in results:
            categories = result.categories
            category_scores = result.category_scores

            # Imprimindo categorias
            print("\nCategorias:")
            print(f"Harassment: {categories.harassment}")
            print(f"Harassment Threatening: {categories.harassment_threatening}")
            print(f"Hate: {categories.hate}")
            print(f"Hate Threatening: {categories.hate_threatening}")
            print(f"Self Harm: {categories.self_harm}")
            print(f"Self Harm Instructions: {categories.self_harm_instructions}")
            print(f"Self Harm Intent: {categories.self_harm_intent}")
            print(f"Sexual: {categories.sexual}")
            print(f"Sexual Minors: {categories.sexual_minors}")
            print(f"Violence: {categories.violence}")
            print(f"Violence Graphic: {categories.violence_graphic}")

            # Imprimindo scores das categorias
            print("\nScores das Categorias:")
            print(f"Harassment: {category_scores.harassment}")
            print(f"Harassment Threatening: {category_scores.harassment_threatening}")
            print(f"Hate: {category_scores.hate}")
            print(f"Hate Threatening: {category_scores.hate_threatening}")
            print(f"Self Harm: {category_scores.self_harm}")
            print(f"Self Harm Instructions: {category_scores.self_harm_instructions}")
            print(f"Self Harm Intent: {category_scores.self_harm_intent}")
            print(f"Sexual: {category_scores.sexual}")
            print(f"Sexual Minors: {category_scores.sexual_minors}")
            print(f"Violence: {category_scores.violence}")
            print(f"Violence Graphic: {category_scores.violence_graphic}")
    else:
        print("Nenhum resultado encontrado na resposta.")

Model:  text-moderation-007

Categorias:
Harassment: False
Harassment Threatening: False
Hate: False
Hate Threatening: False
Self Harm: False
Self Harm Instructions: False
Self Harm Intent: False
Sexual: False
Sexual Minors: False
Violence: True
Violence Graphic: False

Scores das Categorias:
Harassment: 0.07683801651000977
Harassment Threatening: 0.17401666939258575
Hate: 0.006400284823030233
Hate Threatening: 0.00010030140401795506
Self Harm: 0.00039341894444078207
Self Harm Instructions: 1.491239800088806e-06
Self Harm Intent: 0.0003371780621819198
Sexual: 0.00017773699073586613
Sexual Minors: 9.443144790566294e-07
Violence: 0.9916210174560547
Violence Graphic: 4.195504880044609e-05


In [5]:
# https://platform.openai.com/docs/guides/moderation/overview

import requests
import json

url = "https://api.openai.com/v1/moderations"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_api_key}"
}

data = {
    "input": "I want to kill my neighbor"
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    result = response.json()
    print(json.dumps(result, indent=4))
else:
    print(f"Erro: {response.status_code}")
    print(response.text)

{
    "id": "modr-ABT3UaBuc7qpHeDDsWgdi1He0arcd",
    "model": "text-moderation-007",
    "results": [
        {
            "flagged": true,
            "categories": {
                "sexual": false,
                "hate": false,
                "harassment": false,
                "self-harm": false,
                "sexual/minors": false,
                "hate/threatening": false,
                "violence/graphic": false,
                "self-harm/intent": false,
                "self-harm/instructions": false,
                "harassment/threatening": false,
                "violence": true
            },
            "category_scores": {
                "sexual": 0.00017773699073586613,
                "hate": 0.006400284823030233,
                "harassment": 0.07683801651000977,
                "self-harm": 0.00039341894444078207,
                "sexual/minors": 9.443144790566294e-07,
                "hate/threatening": 0.00010030140401795506,
                "violence/gra

In [7]:
from langchain.chains import OpenAIModerationChain 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

moderate = OpenAIModerationChain()
prompt = PromptTemplate.from_template(
 "Esreva sobre {question} \n Responda em português"
)

llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key=openai_api_key)
llm_chain = prompt | llm | StrOutputParser()
moderation_chain = llm_chain | moderate

In [8]:
response = llm_chain.invoke({"question":"Como matar alguém?" })
generated_text = response
print(generated_text)

Sinto muito, mas não posso ajudar com isso.


In [9]:
moderation_response = moderation_chain.invoke({"question":"Como matar alguém?"})
generated_text = moderation_response['output']
print(generated_text)

Sinto muito, mas não posso ajudar com esse tipo de solicitação.


In [10]:
moderation_response

{'input': 'Sinto muito, mas não posso ajudar com esse tipo de solicitação.',
 'output': 'Sinto muito, mas não posso ajudar com esse tipo de solicitação.'}

In [11]:
moderation_response = moderation_chain.invoke({"question": "Ele é estúpido"})
print(moderation_response['output'])

"Ele é estúpido" é uma afirmação que pode ser interpretada de várias maneiras, dependendo do contexto. Às vezes, as pessoas usam esse tipo de expressão para descrever ações ou comportamentos que consideram pouco inteligentes ou sensatos. No entanto, é importante lembrar que chamar alguém de estúpido pode ser muito ofensivo e não contribui para uma comunicação saudável.

Em vez de rotular alguém, pode ser mais construtivo abordar a situação de uma forma mais empática. Tentar entender o porquê de algumas ações ou decisões pode levar a conversas mais produtivas e menos conflitos. Cada um tem suas próprias experiências e formas de ver o mundo, e isso pode influenciar seu comportamento.

Se você está se sentindo frustrado com alguém, pode ser útil expressar seus sentimentos de forma clara e respeitosa, em vez de recorrer a insultos. A comunicação aberta e honesta tende a ser mais eficaz.


In [ ]:
response = llm_chain.invoke({"question": "Ele é estúpido"})
print(response)

In [12]:
moderation_response = moderation_chain.invoke({"question": "qual a capital do brasil ?"})
print(moderation_response['output'])

A capital do Brasil é Brasília. Inaugurada em 21 de abril de 1960, Brasília foi planejada e construída com o objetivo de promover o desenvolvimento do interior do país e reduzir a concentração populacional nas cidades costeiras, como Rio de Janeiro e São Paulo. A cidade foi projetada pelo arquiteto Oscar Niemeyer e pelo urbanista Lúcio Costa, e é conhecida por sua arquitetura modernista e seu planejamento urbano inovador. Brasília é também sede dos principais órgãos do governo federal, incluindo o Palácio do Planalto, o Congresso Nacional e o Supremo Tribunal Federal.


In [ ]:
O modelo da OpenAI (como GPT-3.5 ou GPT-4) já aplica automaticamente mecanismos de moderação para filtrar e ajustar respostas, 
especialmente em relação a conteúdo inapropriado. Portanto, mesmo sem usar explicitamente o OpenAIModerationChain, o sistema 
pode ajustar ou omitir respostas que considere violar as diretrizes.

O OpenAIModerationChain serve mais como uma camada adicional de controle, permitindo que você cheque de forma explícita 
se o conteúdo gerado por um modelo foi sinalizado como inadequado.

Se você já percebeu que as respostas estão sendo moderadas diretamente pelo LLM, pode não ser necessário usar o 
OpenAIModerationChain para moderação básica. No entanto, se você quiser uma moderação explícita para monitorar ou 
registrar quando o conteúdo gerado é considerado inadequado, o uso do OpenAIModerationChain ainda pode ser útil.